In [9]:
import folium
import pandas

In [11]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [13]:
import folium
import pandas as pd

# Define the dataset (mock dataset for this example)
data = {
    'FlightNumber': [1, 2, 3, 4, 5],
    'LaunchSite': ['CCAFS SLC 40', 'CCAFS SLC 40', 'CCAFS SLC 40', 'VAFB SLC 4E', 'CCAFS SLC 40'],
    'Longitude': [-80.577366, -80.577366, -80.577366, -120.610829, -80.577366],
    'Latitude': [28.561857, 28.561857, 28.561857, 34.632093, 28.561857]
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Extract unique launch sites
launch_sites = df[['LaunchSite', 'Longitude', 'Latitude']].drop_duplicates()

# Create a map centered at an average location (mean of latitudes and longitudes)
m = folium.Map(location=[launch_sites['Latitude'].mean(), launch_sites['Longitude'].mean()], zoom_start=4)

# Add markers for each launch site
for _, site in launch_sites.iterrows():
    folium.Marker(
        location=[site['Latitude'], site['Longitude']],
        popup=site['LaunchSite'],
        icon=folium.Icon(color='blue', icon='rocket')
    ).add_to(m)

# Display the map
m


In [19]:
import requests
import pandas as pd
import io

# URL of the CSV file
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'

# Fetch the CSV file from the URL
response = requests.get(URL)

# Read the CSV into a pandas DataFrame
spacex_df = pd.read_csv(io.StringIO(response.text))

# Display the first few rows of the DataFrame
print(spacex_df.head())


   Flight Number        Date Time (UTC) Booster Version  Launch Site  \
0              1  2010-06-04   18:45:00  F9 v1.0  B0003  CCAFS LC-40   
1              2  2010-12-08   15:43:00  F9 v1.0  B0004  CCAFS LC-40   
2              3  2012-05-22    7:44:00  F9 v1.0  B0005  CCAFS LC-40   
3              4  2012-10-08    0:35:00  F9 v1.0  B0006  CCAFS LC-40   
4              5  2013-03-01   15:10:00  F9 v1.0  B0007  CCAFS LC-40   

                                             Payload  Payload Mass (kg)  \
0               Dragon Spacecraft Qualification Unit                0.0   
1  Dragon demo flight C1, two CubeSats,  barrel o...                0.0   
2                             Dragon demo flight C2+              525.0   
3                                       SpaceX CRS-1              500.0   
4                                       SpaceX CRS-2              677.0   

       Orbit         Customer        Landing Outcome  class        Lat  \
0        LEO           SpaceX  Failure   (

In [22]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [24]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [26]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [28]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label


In [30]:
import folium
from folium.features import DivIcon

# Use the existing dataframe
# spacex_df contains the columns: 'Launch Site', 'Lat', 'Long', 'class'

# Start location: NASA Johnson Space Center (as a reference point)
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# For each launch site, add a circle marker based on its coordinate (Lat, Long) and class (success/failure)
for index, row in spacex_df.iterrows():
    # Determine the color based on the launch outcome
    if row['class'] == 1:
        color = 'green'  # Success
        popup_text = f"Launch site: {row['Launch Site']}<br>Outcome: Success"
    else:
        color = 'red'    # Failure
        popup_text = f"Launch site: {row['Launch Site']}<br>Outcome: Failure"
    
    # Add circle for the launch site
    folium.Circle(
        location=[row['Lat'], row['Long']],
        radius=1000,
        color=color,
        fill=True,
        fill_color=color,
        popup=popup_text
    ).add_to(site_map)

    # Add a marker with the site's name
    folium.map.Marker(
        [row['Lat'], row['Long']],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12; color:{color};"><b>{row["Launch Site"]}</b></div>',
        )
    ).add_to(site_map)

# Display the map
site_map


In [32]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [34]:
marker_cluster = MarkerCluster()


In [36]:
from folium.plugins import MarkerCluster

# Function to assign marker color based on class (1: success, 0: failure)
def assign_marker_color(launch_class):
    if launch_class == 1:
        return 'green'
    else:
        return 'red'

# Create a new column `marker_color` in spacex_df based on the class column
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)

# Initialize a map centered on NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Initialize MarkerCluster
marker_cluster = MarkerCluster().add_to(site_map)

# Add markers for each launch site into the marker cluster
for index, row in spacex_df.iterrows():
    # Create a marker for each launch site with its respective color and add it to the cluster
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=f"Launch site: {row['Launch Site']}<br>Outcome: {'Success' if row['class'] == 1 else 'Failure'}",
        icon=folium.Icon(color=row['marker_color'])
    ).add_to(marker_cluster)

# Display the map
site_map


In [38]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map

In [40]:
import numpy as np

# Function to calculate Haversine distance between two points
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth radius in kilometers
    
    # Convert decimal degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    # Differences in coordinates
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Haversine formula
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    # Distance in kilometers
    distance = R * c
    return distance

# Define a target proximity (for example, the Kennedy Space Center)
# Coordinates: 28.573255, -80.646895 (Latitude, Longitude)
target_lat = 28.573255
target_lon = -80.646895

# Calculate distance from each launch site to the proximity
spacex_df['distance_to_proximity_km'] = spacex_df.apply(
    lambda row: haversine(row['Lat'], row['Long'], target_lat, target_lon), axis=1
)

# Display the DataFrame with the new column
print(spacex_df[['Launch Site', 'Lat', 'Long', 'distance_to_proximity_km']])


     Launch Site        Lat        Long  distance_to_proximity_km
0    CCAFS LC-40  28.562302  -80.577356                  6.899285
1    CCAFS LC-40  28.562302  -80.577356                  6.899285
2    CCAFS LC-40  28.562302  -80.577356                  6.899285
3    CCAFS LC-40  28.562302  -80.577356                  6.899285
4    CCAFS LC-40  28.562302  -80.577356                  6.899285
5    CCAFS LC-40  28.562302  -80.577356                  6.899285
6    CCAFS LC-40  28.562302  -80.577356                  6.899285
7    CCAFS LC-40  28.562302  -80.577356                  6.899285
8    CCAFS LC-40  28.562302  -80.577356                  6.899285
9    CCAFS LC-40  28.562302  -80.577356                  6.899285
10   CCAFS LC-40  28.562302  -80.577356                  6.899285
11   CCAFS LC-40  28.562302  -80.577356                  6.899285
12   CCAFS LC-40  28.562302  -80.577356                  6.899285
13   CCAFS LC-40  28.562302  -80.577356                  6.899285
14   CCAFS

In [42]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [44]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [46]:
import folium
from folium.features import DivIcon
from folium.plugins import MousePosition
from math import sin, cos, sqrt, atan2, radians

# Reuse your function for calculating distance between two coordinates
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0  # approximate radius of the Earth in kilometers

    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

# Define launch site coordinates (example)
launch_site_lat = 28.56342
launch_site_lon = -80.57682

# Define closest coastline coordinates found using MousePosition
coastline_lat = 28.56367
coastline_lon = -80.57163

# Create a folium map
nasa_coordinate = [launch_site_lat, launch_site_lon]
site_map = folium.Map(location=nasa_coordinate, zoom_start=15)

# Add MousePosition to get coordinates on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)
site_map.add_child(mouse_position)

# Calculate the distance between the launch site and coastline
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Create and add a marker for the closest coastline point
coastline_marker = folium.Marker(
    location=[coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
    )
)
site_map.add_child(coastline_marker)

# Draw a PolyLine between the launch site and the coastline
coordinates = [[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]]
lines = folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

# TODO: Similarly, draw lines to other proximity points such as the closest city, railway, or highway
# Example for closest city
city_lat = 28.3922  # Example city latitude
city_lon = -80.6077  # Example city longitude
distance_city = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)

# Add city marker
city_marker = folium.Marker(
    location=[city_lat, city_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#2ecc71;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city),
    )
)
site_map.add_child(city_marker)

# Draw PolyLine to the city
city_line = folium.PolyLine(locations=[[launch_site_lat, launch_site_lon], [city_lat, city_lon]], weight=1)
site_map.add_child(city_line)

# Display the map
site_map
